# Instructions

Read the assignment instructions here: https://github.com/markriedl/coinrun-game-ai-assignment

Those new to pytorch may find this [primer](https://colab.research.google.com/drive/1DgkVmi6GksWOByhYVQpyUB4Rk3PUq0Cp) helpful.

Those new to convolutional neural networks may find this [example](https://colab.research.google.com/drive/1740OwEI4oi5QlPtq4UFd3ha44yxef7gU) helpful.

# Installation

Run the following cells

In [ ]:
import os
#del os.environ['LD_PRELOAD']
#!apt-get remove libtcmalloc*

In [ ]:
!apt-get update
!apt-get install mpich build-essential qt5-default pkg-config

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,150 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,334 kB]
Fetched 4,823 kB i

In [ ]:
!git clone https://github.com/markriedl/coinrun-game-ai-assignment.git

Cloning into 'coinrun-game-ai-assignment'...
remote: Enumerating objects: 691, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 691 (delta 21), reused 1 (delta 0), pack-reused 646
Receiving objects: 100% (691/691), 37.74 MiB | 20.72 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [ ]:
!pip install -r coinrun-game-ai-assignment/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 87.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 257.7 kB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654650 sha256=9b7410d7bca233381d87b871b6ad97b38549119729a5fce85dcb9943937cd7fd
  Stored in directory: /root/.cache/pip/wheels/af/4b/74/fcfc8

In [ ]:
import sys
sys.path.insert(0, 'coinrun-game-ai-assignment')

In [ ]:
import torch
torch.cuda.is_available()

True

If the prior cell repots ```False```, then use notebooks settings to turn GPU support on.

In [ ]:
# THIS TESTS THE COINRUN INSTALLATION
from coinrun.random_agent import random_agent

random_agent(max_steps=10)

Logging to /tmp/openai-2023-04-29-20-47-23-255815
step 0 rews [0.]
step 1 rews [0.]
step 2 rews [0.]
step 3 rews [0.]
step 4 rews [0.]
step 5 rews [0.0999999]
step 6 rews [0.0999999]
step 7 rews [0.0999999]
step 8 rews [0.0999999]
step 9 rews [0.0999999]


# Setup

In [ ]:
### RUN THIS BUT DO NOT EDIT THIS CELL
def in_ipynb():
  try:
    result = get_ipython().__class__.__name__
    if 'Shell' in result:
      return True
    else:
      return False
  except:
    return False

IN_PYNB = in_ipynb()



In [ ]:
### RUN THIS BUT DO NOT EDIT THIS CELL
import gym
import os
import math
import random
import numpy as np
from collections import namedtuple
from itertools import count
from PIL import Image

from coinrun import setup_utils, make
import coinrun.main_utils as utils
from coinrun.config import Config
if not IN_PYNB:
    from gym.envs.classic_control import rendering
from coinrun import policies, wrappers

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import pdb

import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

import PIL
from PIL import Image
import shutil

In [ ]:
### RUN THIS BUT DO NOT EDIT THIS CELL

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Resize the screen to this
RESIZE_CONST = 40 


# Game seed information
NUM_LEVELS = 1 # repeat the same level over and over
EASY_LEVEL = 1 # Start on a very small map, no enemies
EASY_LEVEL2 = 5 # Very small map, no enemies
MEDIUM_LEVEL = 20 # Medium length, no enemies
MEDIUM_LEVEL2 = 45 # Medium length, no enemies
ONE_MONSTER = 10 # Short map with one monster
HARD_LEVEL = 7 # Longer and with monsters
LAVA_LEVEL = 3 # Longer and with lava and pits

# Defaults
RENDER_SCREEN = False
SAVE_FILENAME = 'saved.model'
LOAD_FILENAME = 'saved.model'
MODEL_PATH = 'saved_models' 
SEED = EASY_LEVEL

# Don't play with this
EVAL_EPSILON = 0.1
EVAL_WINDOW_SIZE = 3
EVAL_COUNT = 10
TIMEOUT = 1000
COIN_REWARD = 100

### Data structure for holding experiences for replay
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

### Function for resizing the screen
resize = T.Compose([T.ToPILImage(),
                    T.Resize(RESIZE_CONST, interpolation=Image.CUBIC),
                    T.ToTensor()])

### Save the model. Extra information can be added to the end of the filename
def save_model(model, filename, extras = None):
    if extras is not None:
        filename = filename + '.' + str(extras)
    print("Saving", filename, "...")
    torch.save(model, os.path.join(MODEL_PATH, filename))
    print("Done saving.")
 
### Load the model. If there are multiple versions with extra information at the
### end of the filename, get the latest.
def load_model(filename, extras = None):
    if extras is not None:
        filename = filename + '.' + str(extras)
    model = None
    candidates = [os.path.join(MODEL_PATH, f) for f in os.listdir(MODEL_PATH) if filename in f]
    if len(candidates) > 0:
        candidates = sorted(candidates, key=lambda f:os.stat(f).st_mtime, reverse=True)
        filename = candidates[0]
        print("Loading", filename, "...")
        model = torch.load(filename)
        print("Done loading.")
    return model
  
### Give a text description of the outcome of an episode and also a score
### Score is duration, unless the agent died.
def episode_status(duration, reward):
    status = ""
    score = 0
    if duration >= TIMEOUT:
        status = "timeout"
        score = duration
    elif reward < COIN_REWARD:
        status = "died"
        score = TIMEOUT
    else:
        status = "coin"
        score = duration
    return status, score

Every time the code runs an ```evaluate()``` of the agent's model, a new directory will be created in ```/content/cache``` and screenshots of the agent in action will be stored. Filenames are of the form ```/content/cache/eval_number/eval + screen_number + .jpeg```.

In [ ]:
RUN_NUM = 0
SCREEN_SAVE = True
SCREEN_COUNT = 0
SCREEN_SAVE_PREFIX = 'eval'
SCREEN_SAVE_POSTFIX = '.jpeg'
TEMP_DIR = 'cache'
SCREEN_SAVE_RATIO = 0.5

#Screen is numpy array
def save_screen(screen):
  global SCREEN_COUNT
  screen_array_t = np.transpose(screen, (1, 2, 0))
  img = Image.fromarray(np.uint8(screen_array_t * 255))
  width, height = img.size
  img = img.resize((int(width*SCREEN_SAVE_RATIO), int(height*SCREEN_SAVE_RATIO)), Image.LANCZOS)
  if not os.path.isdir(TEMP_DIR):
    os.mkdir(TEMP_DIR)
  if not os.path.isdir(os.path.join(TEMP_DIR, str(RUN_NUM))):
    os.mkdir(os.path.join(TEMP_DIR, str(RUN_NUM)))
  img.save(os.path.join(TEMP_DIR, str(RUN_NUM), SCREEN_SAVE_PREFIX + str(SCREEN_COUNT) + SCREEN_SAVE_POSTFIX), "JPEG")
  SCREEN_COUNT = SCREEN_COUNT + 1

def show_movie(num):
  dir_name = str(num)
  files = []
  for file in os.listdir(os.path.join(TEMP_DIR, dir_name)):
    files.append(file)
  sorted_files = sorted(files, key=lambda f:int(f[len(SCREEN_SAVE_PREFIX):-len(SCREEN_SAVE_POSTFIX)]))
  for file in sorted_files:
    img = Image.open(os.path.join(TEMP_DIR, dir_name, file), 'r')
    plt.imshow(img)
    ipythondisplay.clear_output(wait=True)
    ipythondisplay.display(plt.gcf())

def make_anim(num):
  dir_name = str(num)
  files = []
  images = []
  for file in os.listdir(os.path.join(TEMP_DIR, dir_name)):
    files.append(file)
  sorted_files = sorted(files, key=lambda f:int(f[len(SCREEN_SAVE_PREFIX):-len(SCREEN_SAVE_POSTFIX)]))
  for file in sorted_files:
    try:
      img = Image.open(os.path.join(TEMP_DIR, dir_name, file))
      images.append(img)
    except:
      print(os.path.join(TEMP_DIR, dir_name, file), "did not load.")
  images[0].save('movie' + str(num) + '.gif', "GIF",
                      save_all=True,
                      append_images=images[1:],
                      duration=1)


# Globals

In [ ]:
# YOU MAY EDIT THESE IF NECESSARY

BATCH_SIZE = 128            # How many replay experiences to run through neural net at once
GAMMA = 0.999               # How much to discount the future [0..1]
BOOTSTRAP = 10000           # How many steps to run to fill up replay memory before training starts
TARGET_UPDATE = 0           # Delays updating the network for loss calculations. 0=don't delay, or 1+ number of episodes
REPLAY_CAPACITY = 10000     # How big is the replay memory
EPSILON = 0.9               # Use random action if less than epsilon [0..1]
EVAL_INTERVAL = 10          # How many episodes of training before evaluation
NUM_EPISODES = 500          # Max number of training episodes
RANDOM_SEED = None          # Seed for random number generator, for reproducability, use None for random seed

# Reference Functions

Run these cells but do not edit them.

## Unit Testing Functions

In [ ]:
# RUN THIS CELL BUT DO NOT EDIT.

def testReplayMemory():
    print("Testing ReplayMemory...")
    capacity = 100
    test_replay_memory = ReplayMemory(capacity)
    for i in range(capacity):
        test_replay_memory.push(i, i, i, i)
    assert (len(test_replay_memory) == capacity),"size test failed"
    for i in range(len(test_replay_memory)):
        item = test_replay_memory.memory[i]
        assert (item[0] == i), "item" + str(i) + "not holding the correct value"
    for i in range(capacity//2):
        test_replay_memory.push(capacity+i, capacity+i, capacity+i, capacity+i)
    assert (len(test_replay_memory) == capacity), "size test 2 failed"
    # check items
    for i in range(len(test_replay_memory)):
        item = test_replay_memory.memory[i]
        if i < capacity // 2:
            assert (item[0] == i+capacity), "not holding the correct value after looping (first half)"
        else:
            assert (item[0] == i), "not holding the correct value after looping (second half)"
    print("ReplayMemory test passed.")
    return True
  
def testMakeBatch():
    print("Testing doMakeBatch...")
    batch_size = 128
    capacity = batch_size * 2
    test_replay_memory = ReplayMemory(capacity)
    state = None
    new_state = None
    action = None
    reward = None
    # Test types and shapes of return values
    for i in range(capacity):
        state = torch.randn(1, 3, 80, 80, device=DEVICE)
        new_state = torch.randn(1, 3, 80, 80, device=DEVICE)
        action = torch.randn(1, 1, device=DEVICE)
        reward = torch.randn(1, 1, device=DEVICE)
        test_replay_memory.push(state, action, new_state, reward)
    states_batch, actions_batch, next_states_batch, rewards_batch, non_final_mask = doMakeBatch(test_replay_memory, batch_size)
    assert(type(states_batch) == torch.Tensor and states_batch.size() == (batch_size, 3, 80, 80)), "states batch not correct shape."
    assert(type(actions_batch) == torch.Tensor and actions_batch.size() == (batch_size, 1)), "actions batch not correct shape."
    assert(type(next_states_batch) == torch.Tensor and next_states_batch.size() == (batch_size, 3, 80, 80)), "next states batch not correct shape."
    assert(type(rewards_batch) == torch.Tensor and rewards_batch.size() == (batch_size, 1)), "rewards batch not correct shape."
    assert(type(non_final_mask) == type(torch.tensor(batch_size, dtype=torch.bool, device=DEVICE)) and non_final_mask.size()[0] == batch_size), "non-final mask not correct shape."

    # Test mask
    test_replay_memory = ReplayMemory(batch_size)
    for i in range(batch_size):
        state = torch.randn(1, 3, 80, 80, device=DEVICE)
        new_state = None
        if i % 2 == 0:
            new_state = torch.randn(1, 3, 80, 80, device=DEVICE)
        action = torch.randn(1, 1, device=DEVICE)
        reward = torch.randn(1, 1, device=DEVICE)
        test_replay_memory.push(state, action, new_state, reward)
    states_batch, actions_batch, next_states_batch, rewards_batch, non_final_mask = doMakeBatch(test_replay_memory, batch_size)
    assert(non_final_mask.sum() == batch_size//2), "non_final_mask not masking properly."
    print("doMakeBatch test passed.")
    return True

class UnitTestDQN(nn.Module):
    def __init__(self, h, w, num_actions):
        super(UnitTestDQN, self).__init__()
        self.num_actions = num_actions
    def forward(self, x):
        assert(False), "Network should not be queried when epsilon = 1.0." 
        return None

def testSelectAction():
    print("Testing select_action...")
    from scipy.stats import chisquare
    sample_size = 10000
    num_tests = 100
    pass_rate = 0.9
    screen_height = 40
    screen_width = 40
    epsilon = 1.0
    num_actions = 7
    test_results = {True: 0, False: 0}
    significance_level = 0.02
    net = UnitTestDQN(screen_height, screen_width, num_actions).to(DEVICE)
    state = torch.randn(1, 3, 80, 80, device=DEVICE)
    for j in range(num_tests):
        samples = {}
        for i in range(sample_size):
            action, new_epsilon = select_action(state, net, num_actions, epsilon, steps_done = 0, bootstrap_threshold = 2)
            assert(type(action) == torch.Tensor and action.size() == (1,1)), "Action not correct shape."
            assert(new_epsilon == epsilon), "Epsilon should not change during bootstrapping."
            action = action.item()
            if action not in samples:
                samples[action] = 0
            samples[action] = samples[action] + 1
        expected = [sample_size / num_actions] * num_actions
        statistic, pvalue = chisquare(f_obs=list(samples.values()), f_exp=expected)
        test_results[pvalue >= significance_level] += 1
    assert(test_results[True] > pass_rate * num_tests), "Random sample is not from uniform distribution."    
    print("select_action test passed.")
    return True

def testPredictQValues():
    print("Testing doPredictQValues...")
    batch_size = 128
    screen_height = 80
    screen_width = 80
    num_actions = 7
    net = DQN(screen_height, screen_width, num_actions).to(DEVICE)
    states_batch = torch.randn(batch_size, 3, 80, 80, device=DEVICE)
    actions_batch = torch.randint(0, 7, (128, 1), device=DEVICE).long()
    state_action_values = doPredictQValues(net, states_batch, actions_batch)
    assert(type(state_action_values) == torch.Tensor and state_action_values.size() == (128, 1)), "Return value not correct shape."
    print("doPredictQValues test passed.")
    return True

def testPredictNextStateUtilities():
    print("Testing doPredictNextStateUtilities...")
    screen_height = 80
    screen_width = 80
    num_actions = 7
    batch_size = 128
    passed = False
    net = DQN(screen_height, screen_width, num_actions).to(DEVICE)
    # First option to try is that the batch is full sized.
    try:
        next_states_batch = torch.ones(batch_size, 3, 80, 80, device=DEVICE)
        non_final_mask = torch.ones(batch_size, dtype=torch.bool, device=DEVICE)
        for i in range(batch_size):
            if i % 2 == 1:
                next_states_batch[i].fill_(0)
                non_final_mask[i] = 0
        next_state_values = doPredictNextStateUtilities(net, next_states_batch, non_final_mask, batch_size)
        assert(type(next_state_values) == torch.Tensor and next_state_values.size() == (batch_size, 1)), "Return value not correct shape (attempt 1)."
        for i in range(batch_size):
            if i % 2 == 1:
                assert(next_state_values[i].sum() == 0), "Element " + str(i) + "is not 0.0 when non_final_mask[i] = 0"
        passed = True
    except RuntimeError as e:
        print(e)
        print("Will try alternative test.")
    if not passed:
        # Next option is that batch is not full sized.
        try:
            next_states_batch = torch.ones(batch_size-1, 3, 80, 80, device=DEVICE)
            non_final_mask = torch.ones(batch_size, dtype=torch.bool, device=DEVICE)
            non_final_mask[0] = 0
            next_state_values = doPredictNextStateUtilities(net, next_states_batch, non_final_mask, batch_size)
            assert(type(next_state_values) == torch.Tensor and next_state_values.size() == (batch_size, 1)), "Return value not correctd shape (attempt 2)."
            passed = True
        except RuntimeError as e:
            print(e)
            print("No further alternative tests available.")
    if passed:
        print("doPredictNextStateUtilities test passed.")
        return True
    assert(False), "doPredictNextStateUtilities did NOT pass test."

def testComputeExpectedQValues():
    print("Testing doComputeExpectedQValues...")
    batch_size = 128
    gamma = 0.5
    next_state_values = torch.ones(batch_size).unsqueeze(1)
    rewards_batch = torch.ones(batch_size).unsqueeze(1)
    expected_state_action_values = doComputeExpectedQValues(next_state_values, rewards_batch, gamma)
    assert(type(expected_state_action_values) == torch.Tensor and expected_state_action_values.size()[0] == batch_size), "Return value not expected shape."
    for i in range(batch_size):
        assert(expected_state_action_values[i].item() == 1.5), "Element " + str(i) + " doesn't have the correct value."
    print("doComputeExpectedQValues test passed.")
    return True

def testComputeLoss():
    print("Testing doComputeLoss...")
    batch_size = 128
    state_action_values = torch.randn(batch_size, device=DEVICE)
    expected_state_action_values = torch.randn(batch_size, device=DEVICE)
    loss = doComputeLoss(state_action_values, expected_state_action_values)
    assert(type(loss) == torch.Tensor and len(loss.size()) == 0), "Loss not of expected shape."
    print("doComputeLoss test passed.")
    return True


def unit_test():
    testReplayMemory()
    testMakeBatch()
    testSelectAction()
    testPredictQValues()
    testPredictNextStateUtilities()
    testComputeExpectedQValues()
    testComputeLoss()

## Training loop

Read this code. It will be helpful to know how the functions you will be modifying are used.

In [ ]:
# RUN THIS CELL BUT DO NOT EDIT.

### Training loop.
### Each episode is a game that runs until the agent gets the coin or the game times out.
### Train for a given number of episodes.
def train(num_episodes = NUM_EPISODES, load_filename = None, save_filename = None, eval_interval = EVAL_INTERVAL, replay_capacity = REPLAY_CAPACITY, bootstrap_threshold = BOOTSTRAP, epsilon = EPSILON, eval_epsilon = EVAL_EPSILON, gamma = GAMMA, batch_size = BATCH_SIZE, target_update = TARGET_UPDATE, random_seed = RANDOM_SEED, num_levels = NUM_LEVELS, seed = SEED):
    # Set the random seed
    if random_seed is not None:
        random.seed(random_seed)
        torch.manual_seed(random_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(RANDOM_SEED)
    # Set up the environment
    setup_utils.setup_and_load(use_cmd_line_args=False, is_high_res=True, num_levels=num_levels, set_seed=seed)
    env = make('standard', num_envs=1)
    if RENDER_SCREEN and not IN_PYNB:
        env.render()

    # Reset the environment
    env.reset()

    # Get screen size so that we can initialize layers correctly based on shape returned from AI gym. 
    init_screen = get_screen(env)
    _, _, screen_height, screen_width = init_screen.shape
    print("screen size: ", screen_height, screen_width)

    # Are we resuming from an existing model?
    policy_net = None
    if load_filename is not None and os.path.isfile(os.path.join(MODEL_PATH, load_filename)):
        print("Loading model...")
        policy_net = load_model(load_filename)
        policy_net = policy_net.to(DEVICE)
        print("Done loading.")
    else:
        print("Making new model.")
        policy_net = DQN(screen_height, screen_width, env.NUM_ACTIONS).to(DEVICE)
    # Make a copy of the policy network for evaluation purposes
    eval_net = DQN(screen_height, screen_width, env.NUM_ACTIONS).to(DEVICE)
    eval_net.load_state_dict(policy_net.state_dict())
    eval_net.eval()
    # Target network is a snapshot of the policy network that lags behind (for stablity)
    target_net = DQN(screen_height, screen_width, env.NUM_ACTIONS).to(DEVICE)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()
    
    # Instantiate the optimizer
    optimizer = None
    if len(list(policy_net.parameters())) > 0:
        optimizer = initializeOptimizer(policy_net.parameters())
    
    # Instantiate the replay memory
    replay_memory = ReplayMemory(replay_capacity)

    steps_done = 0               # How many steps have been run
    best_eval = float('inf')     # The best model evaluation to date

    ### Do training until episodes complete 
    print("training...")
    i_episode = 0            # The episode number
    
    # Stop when we reach max episodes
    while i_episode < num_episodes:
        print("episode:", i_episode, "epsilon:", epsilon)
        max_reward = 0       # The best reward we've seen this episode
        done = False         # Has the game ended (timed out or got the coin)
        episode_steps = 0    # Number of steps performed in this episode
        # Initialize the environment and state
        env.reset()
        
        # Current screen. There is no last screen because we get velocity on the screen itself.
        state = get_screen(env)

        # Do forever until the loop breaks 
        while not done:
            # Select and perform an action
            action, epsilon = select_action(state, policy_net, env.NUM_ACTIONS, epsilon, steps_done, bootstrap_threshold)
            steps_done = steps_done + 1
            episode_steps = episode_steps + 1
            
            # for debugging
            if RENDER_SCREEN and not IN_PYNB:
                env.render() 

            # Run the action in the environment
            if action is not None: 
                _, reward, done, _ = env.step(np.array([action.item()]))

                # Record if this was the best reward we've seen so far
                max_reward = max(reward, max_reward)
                
                # Turn the reward into a tensor  
                reward = torch.tensor([reward], device=DEVICE)

                # Observe new state
                current_screen = get_screen(env)

                # Did the game end?
                if not done:
                    next_state = current_screen
                else:
                    next_state = None

                # Store the transition in memory
                replay_memory.push(state, action, next_state, reward)

                # Move to the next state
                state = next_state

                # If we are past bootstrapping we should perform one step of the optimization
                if steps_done > bootstrap_threshold:
                  optimize_model(policy_net, target_net if target_update > 0 else policy_net, replay_memory, optimizer, batch_size, gamma)
            else:
                # Do nothing if select_action() is not implemented and returning None
                env.step(np.array([0]))
                
            # If we are done, print some statistics
            if done:
                print("duration:", episode_steps)
                print("max reward:", max_reward)
                status, _ = episode_status(episode_steps, max_reward)
                print("result:", status)
                print("total steps:", steps_done, '\n')

            # Should we update the target network?
            if target_update > 0 and i_episode % target_update == 0:
                target_net.load_state_dict(policy_net.state_dict())
                
        # Should we evaluate?
        if steps_done > bootstrap_threshold and i_episode > 0 and i_episode % eval_interval == 0:
            test_average_duration = 0       # Track the average eval duration
            test_average_max_reward = 0     # Track the average max reward
            # copy all the weights into the evaluation network
            eval_net.load_state_dict(policy_net.state_dict())
            # Evaluate 10 times
            for _ in range(EVAL_COUNT):
                # Call the evaluation function
                test_duration, test_max_reward = evaluate(eval_net, eval_epsilon, env, test_seed=seed)
                status, score = episode_status(test_duration, test_max_reward)
                test_duration = score # Set test_duration to score to factor in death-penalty
                test_average_duration = test_average_duration + test_duration
                test_average_max_reward = test_average_max_reward + test_max_reward
            test_average_duration = test_average_duration / EVAL_COUNT
            test_average_max_reward = test_average_max_reward / EVAL_COUNT
            print("Average duration:", test_average_duration)
            print("Average max reward:", test_average_max_reward)
            # If this is the best window average we've seen, save the model
            if test_average_duration < best_eval:
                best_eval = test_average_duration
                if save_filename is not None:
                    save_model(policy_net, save_filename, i_episode)
            print(' ')
        # Only increment episode number if we are done with bootstrapping
        if steps_done > bootstrap_threshold:
          i_episode = i_episode + 1
    print('Training complete')
    if RENDER_SCREEN and not IN_PYNB:
        env.render()
    env.close()
    return policy_net


## Optimization Function

Read this code. It will be helpful to know how the functions you will be modifying are used.

In [ ]:
# RUN THIS CELL BUT DO NOT EDIT.

### Take a DQN and do one forward-backward pass.
### Since this is Q-learning, we will run a forward pass to get Q-values for state-action pairs and then 
### give the true value as the Q-values after the Q-update equation.
def optimize_model(policy_net, target_net, replay_memory, optimizer, batch_size, gamma):
    if len(replay_memory) < batch_size:
        return
    ### step 1: sample from the replay memory. Get BATCH_SIZE transitions
    ### Step 2: Get a list of non-final next states.
    ###         a. Create a mask, a tensor of length BATCH_SIZE where each element i is 1 if 
    ###            batch.next_state[i] is not None and 0 otherwise.
    ###         b. Create a tensor of shape [BATCH_SIZE, color(3), height, width] by concatenating
    ###            all non-final (not None) batch.next_states together.
    ### Step 3: set up batches for state, action, and reward
    ###         a. Create a tensor of shape [BATCH_SIZE, color(3), height, width] holding states
    ###         b. Create a tensor of shape [BATCH_SIZE, 1] holding actions
    ###         c. Create a tensor of shape [BATCH_SIZE, 1] holding rewards
    states_batch, actions_batch, next_states_batch, rewards_batch, non_final_mask = doMakeBatch(replay_memory, batch_size)

    ### Step 4: Get the action values predicted.
    ###         a. Call policy_net(state_batch) to get a tensor of shape [BATCH_SIZE, NUM_ACTIONS] containing Q-values
    ###         b. For each batch, get the Q-value for the corresponding action in action_batch (hint: torch.gather)
    state_action_values = doPredictQValues(policy_net, states_batch, actions_batch)

    ### Step 5: Get the utility values of next_states.
    next_state_values = doPredictNextStateUtilities(target_net, next_states_batch, non_final_mask, batch_size)
    
    ### Step 6: Compute the expected Q values.
    expected_state_action_values = doComputeExpectedQValues(next_state_values, rewards_batch, gamma)

    ### Step 7: Computer Huber loss (smooth L1 loss)
    ###         Compare state action values from step 5 to expected state action values from step 7
    loss = doComputeLoss(state_action_values, expected_state_action_values)
    ### Step 8: Back propagation
    ###         a. Zero out gradients
    ###         b. call loss.backward()
    ###         c. Prevent gradient explosion by clipping gradients between -1 and 1
    ###            (hint: param.grad.data is the gradients. See torch.clamp_() )
    ###         d. Tell the optimizer that another step has occurred: optimizer.step()
    if optimizer is not None:
        optimizer.zero_grad()
        doBackprop(loss, policy_net.parameters())
        optimizer.step()

## Evaluation function

In [ ]:
# RUN THIS CELL BUT DO NOT EDIT.

### Evaluate the DQN
### If environment is given, use that. Otherwise make a new environment.
def evaluate(policy_net, epsilon = EVAL_EPSILON, env = None, test_seed = SEED):
    global RUN_NUM
    RUN_NUM = RUN_NUM + 1
    setup_utils.setup_and_load(use_cmd_line_args=False, is_high_res=True, num_levels=NUM_LEVELS, set_seed=test_seed)
    

    # Make an environment if we don't already have one
    if env is None:
        env = make('standard', num_envs=1)
    if RENDER_SCREEN and not IN_PYNB:
        env.render()

    # Reset the environment
    env.reset()

    # Get screen size so that we can initialize layers correctly based on shape
    # returned from AI gym. 
    init_screen = get_screen(env, SCREEN_SAVE)
    _, _, screen_height, screen_width = init_screen.shape

    # Get the network ready for evaluation (turns off some things like dropout if used)
    policy_net.eval()

    # Current screen. There is no last screen
    state = get_screen(env, SCREEN_SAVE)

    steps_done = 0         # Number of steps executed
    max_reward = 0         # Max reward seen
    done = False           # Is the game over?

    print("Evaluating...")
    while not done:
        # Select and perform an action
        action, _ = select_action(state, policy_net, env.NUM_ACTIONS, epsilon, steps_done=0, bootstrap_threshold=0)
        steps_done = steps_done + 1

        if RENDER_SCREEN and not IN_PYNB:
            env.render()          

        # Execute the action
        if action is not None:
            _, reward, done, _ = env.step(np.array([action.item()]))

            # Is this the best reward we've seen?
            max_reward = max(reward, max_reward)

            # Observe new state
            state = get_screen(env, SCREEN_SAVE)
        else:
            # Do nothing if select_action() is not implemented and returning None
            env.step(np.array([0]))

    print("duration:", steps_done)
    print("max reward:", max_reward)
    status, _ = episode_status(steps_done, max_reward)
    print("result:", status, '\n')
    if RENDER_SCREEN and not IN_PYNB:
        env.render()
    return steps_done, max_reward

# Your Implementation

You must complete the following functions.

## DQN

In [ ]:
class DQN(nn.Module):

    ### Create all the nodes in the computation graph.
    ### We won't say how to put the nodes together into a computation graph. That is done
    ### automatically when forward() is called.
    def __init__(self, h, w, num_actions):
        super(DQN, self).__init__()
        self.num_actions = num_actions
        ### WRITE YOUR CODE BELOW HERE
        self.conv1 = torch.nn.Conv2d(3, 10, 5)
        self.batchnorm1 = torch.nn.BatchNorm2d(10)
        self.leaky_relu = torch.nn.LeakyReLU(0.01)
        self.conv2 = torch.nn.Conv2d(10, 30, 5)
        self.batchnorm2 = torch.nn.BatchNorm2d(30)
        self.linear = torch.nn.Linear(30 * (h - 8) * (w - 8), self.num_actions)
        ### WRITE YOUR CODE ABOVE HERE

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        q_values = None
        ### WRITE YOUR CODE BELOW HERE
        conv1_out = self.conv1(x)
        #print("conv1_out: " + str(conv1_out.shape))
        batchnorm1_out = self.batchnorm1(conv1_out)
        #print("batchnorm1_out: " + str(batchnorm1_out.shape))
        activ1_out = self.leaky_relu(batchnorm1_out)
        #print("activ1_out: " + str(activ1_out.shape))
        conv2_out = self.conv2(activ1_out)
        #print("conv2_out: " + str(conv2_out.shape))
        batchnorm2_out = self.batchnorm2(conv2_out)
        #print("batchnorm2_out: " + str(batchnorm2_out.shape))
        activ2_out = self.leaky_relu(batchnorm2_out)
        #print("activ2_out: " + str(activ2_out.shape))
        activ2_out = activ2_out.view(activ2_out.size(0), -1)
        #print("activ2_out: " + str(activ2_out.shape))
        linear_out = self.linear(activ2_out)
        q_values = linear_out
        ### WRITE YOUR CODE ABOVE HERE
        return q_values

## get_screen

In [ ]:
### Take the environment and return a tensor containing screen data as a 3D tensor containing (color, height, width) information.
### Optional: the screen may be manipulated, for example, it could be cropped
def get_screen(env, save = False):
    # Returned screen requested by gym is 512x512x3. Transpose it into torch order (Color, Height, Width).
    screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    _, screen_height, screen_width = screen.shape
    ### DO ANY SCREEN MANIPULATIONS NECESSARY (IF ANY)

    ### END SCREEN MANIPULATIONS
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    if save:
      save_screen(screen)
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return resize(screen).unsqueeze(0).to(DEVICE)

## ReplayMemory

In [ ]:
### Store transitions to use to prevent catastrophic forgetting.
### ReplayMemory implements a ring buffer. Items are placed into memory
###    until memory reaches capacity, and then new items start replacing old items
###    at the beginning of the array. 
### Member variables:
###    capacity: (int) number of transitions that can be stored
###    memory: (array) holds transitions (state, action, next_state, reward)
###    position: (int) index of current location in memory to place the next transition.
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    ### Store a transition in memory.
    ### To implement: put new items at the end of the memory array, unless capacity is reached.
    ###    Combine the arguments into a new Transition object.
    ###    If capacity is reached, start overwriting the beginning of the array.
    ###    Use the position index to keep track of where to put the next item. 
    def push(self, state, action, next_state, reward):
        ### WRITE YOUR CODE BELOW HERE
        if len(self.memory) < self.capacity:
          self.memory.append(Transition(state, action, next_state, reward))
          self.position += 1
        elif len(self.memory) == self.capacity:
          if self.position == self.capacity:
            self.position = 0
          self.memory[self.position] = Transition(state, action, next_state, reward)
          self.position += 1
        ### WRITE YOUR CODE ABOVE HERE

    ### Return a batch of transition objects from memory containing batch_size elements.
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    ### This allows one to call len() on a ReplayMemory object. E.g. len(replay_memory)
    def __len__(self):
        return len(self.memory)


## initializeOptimizer

In [ ]:
### Choose and instantiate an optimizer. A default example is given, which you can change.
### Input:
### - parameters: the DQN parameters
### Output:
### - the optimizer object
def initializeOptimizer(parameters):
    optimizer = None
    ### WRITE YOUR CODE BELOW HERE
    #optimizer = optim.SGD(parameters, lr=0.01)
    optimizer = optim.Adam(parameters, lr = 0.001)
    ### WRITE YOUR CODE ABOVE HERE
    return optimizer

## select_action

In [ ]:
import random

### Select an action to perform. 
### If a random number [0..1] is greater than epsilon, then query the policy_network,
### otherwise use a random action.
### Inputs:
### - state: a tensor of shape 3 x screen_height x screen_width
### - policy_net: a DQN object
### - num_actions: number of actions available
### - epsilon: float [0..1] indicating whether to choose random or use the network
### - steps_done: number of previously executed steps
### - bootstrap_threshold: number of steps that must be executed before training begins
### This function should return:
### - A tensor of shape 1 x 1 that contains the number of the action to execute
### - The new epsilon value to use next time
def select_action(state, policy_net, num_actions, epsilon, steps_done = 0, bootstrap_threshold = 0):
    action = None
    new_epsilon = epsilon
    ### WRITE YOUR CODE BELOW HERE
    rand_num = random.uniform(0, 1)
    if rand_num > epsilon:
      action = policy_net(state).max(1)[1].view(1, 1)
    else:
      action = torch.tensor([[random.randrange(num_actions)]], device=DEVICE, dtype=torch.long)
    if steps_done > bootstrap_threshold:
      new_epsilon = max(0.9 * math.exp(-1.0 * (steps_done - bootstrap_threshold) / 100000), 0.1)
    ### WRITE YOUR CODE ABOVE HERE
    return action, new_epsilon

## doMakeBatch

In [ ]:
### Ask for a batch of experience replays.
### Inputs:
### - replay_memory: A ReplayMemory object
### - batch_size: size of the batch to return
### Outputs:
### - states_batch: a tensor of shape batch_size x 3 x screen_height x screen_width
### - actions_batch: a tensor of shape batch_size x 1 containing action numbers
### - next_states_batch: a tensor containing screens. 
### - rewards_batch: a tensor of shape batch_size x 1 containing reward values.
### - non_final_mask: a tensor of bytes of length batch_size containing a 0 if the state is terminal or 1 otherwise
def doMakeBatch(replay_memory, batch_size):
    states_batch = None
    actions_batch = None
    next_states_batch = None
    rewards_batch = None
    non_final_mask = None
    ### WRITE YOUR CODE BELOW HERE
    sample = Transition(*zip(*replay_memory.sample(batch_size)))
    states_batch = torch.cat(sample.state)
    actions_batch = torch.cat(sample.action)
    next_states_batch = torch.cat([state for state in sample.next_state if state is not None])
    rewards_batch = torch.cat(sample.reward)
    non_final_mask = torch.tensor((tuple(map(lambda x: x is not None, sample.next_state))), device=DEVICE, dtype=(torch.bool))
    #print(non_final_mask)
    ### WRITE YOUR CODE ABOVE HERE
    return states_batch, actions_batch, next_states_batch, rewards_batch, non_final_mask

    #sample = Transition(*zip(*replay_memory.sample(batch_size))) 
    #state batch - concatenate all elements in sample.state - torch.cat(sample.state)
    # same for actions and rewards
    # next_states_batch - concatenate all next states that are not none (sample.next_states)
    # non_final_mask - 0 when state is none and 1 when the state is not none....append 1 or 0 to list and then convert to tensor

## doPredictQValues

In [ ]:
### Ask the policy_net to predict the Q value for a batch of states and a batch of actions.
### Inputs:
### - policy_net: the DQN
### - states_batch: a tensor of shape batch_size x 3 x screen_height x screen_width containing screens
### - actions_batch: a tensor of shape batch_size x 1 containing action numbers
### Output:
### - A tensor of shape batch_size x 1 containing the Q-value predicted by the DQN in the position indicated by the action
def doPredictQValues(policy_net, states_batch, actions_batch):
    state_action_values = None
    ### WRITE YOUR CODE BELOW HERE
    state_action_values = policy_net(states_batch)
    state_action_values = state_action_values.gather(1, actions_batch)
    ### WRITE YOUR CODE ABOVE HERE
    return state_action_values

    #call policy network on given states batch
    #gather that based on action_batch (axis = 1)

## doPredictNextStateUtilities

In [ ]:
### Ask the policy_net to predict the utility of a next_state.
### Inputs:
### - policy_net: The DQN
### - next_states_batch: a tensor of shape batch_size x 3 x screen_height x screen_width
### - non_final_mask: a tensor of length batch_size containing 0 for terminal states and 1 for non-terminal states
### - batch_size: the batch size
### Note: Only run non-terminal states through the policy_net
### Output:
### - A tensor of shape batch_size x 1 containing Q-values
def doPredictNextStateUtilities(policy_net, next_states_batch, non_final_mask, batch_size):
    next_state_values = torch.zeros(batch_size, device=DEVICE)
    ### WRITE YOUR CODE BELOW HERE
    next_state_values[non_final_mask] = policy_net(next_states_batch).max(1)[0]
    next_state_values = next_state_values.unsqueeze(1)
    ### WRITE YOUR CODE ABOVE HERE
    return next_state_values.detach()

    #call the policy net on the next_states batch
    # for each state, provide the policy's result
    # using the mask, set next_state_values[non_final_mask] = policy's best policy (best/max policy across each column -summed) and then unsqueeze

## doComputeExpectedQValues

In [ ]:
### Compute the Q-update equation Q(s_t, a_t) = R(s_t+1) + gamma * argmax_a' Q(s_t+1, a')
### Inputs:
### - next_state_values: a tensor of shape batch_size x 1 containing Q values for state s_t+1
### - rewards_batch: a tensor or shape batch_size x 1 containing reward values for state s_t+1
### Output:
### - A tensor of shape batch_size x 1
def doComputeExpectedQValues(next_state_values, rewards_batch, gamma):
    expected_state_action_values = None
    ### WRITE YOUR CODE BELOW HERE
    expected_state_action_values = torch.add(rewards_batch, torch.mul(gamma, next_state_values))
    ### WRITE YOUR CODE ABOVE HERE
    return expected_state_action_values

    #gamma * next_state values + rewards_batch

## doComputeLoss

In [ ]:
### Compute the loss
### Inputs:
### - state_action_values: a tensor of shape batch_size x 1 containing Q values
### - expected_state_action_values: a tensor of shape batch_size x 1 containing updated Q values
### Output:
### - A tensor scalar value
def doComputeLoss(state_action_values, expected_state_action_values):
    loss = None
    ### WRITE YOUR CODE BELOW HERE
    loss = torch.nn.functional.smooth_l1_loss(state_action_values, expected_state_action_values)
    ### WRITE YOUR CODE ABOVE HERE
    return loss

    #call nn.functional.smooth_l1_loss(state_action_values, expected_state_values)

## doBackprop

In [ ]:
### Run backpropagation. Make sure gradients are clipped between -1 and +1.
### Inputs:
### - loss: a tensor scalar
### - parameters: the parameters of the DQN
### There is no output
def doBackprop(loss, parameters):
    ### WRITE YOUR CODE BELOW HERE
    loss.backward()
    for param in parameters:
      param.grad.data.clamp_(-1, 1)
    ### WRITE YOUR CODE ABOVE HERE

    #loss.backward()
    #go thru each parameter and clip between -1 and 1, use stack overflow link: https://stackoverflow.com/questions/54716377/how-to-do-gradient-clipping-in-pytorch

# Unit testing

Run this cell to make sure your functions return valid outputs.

In [ ]:
### RUN UNIT TESTS ON FUNCTIONS
unit_test() 

Testing ReplayMemory...
ReplayMemory test passed.
Testing doMakeBatch...
doMakeBatch test passed.
Testing select_action...
select_action test passed.
Testing doPredictQValues...
doPredictQValues test passed.
Testing doPredictNextStateUtilities...
shape mismatch: value tensor of shape [128] cannot be broadcast to indexing result of shape [64]
Will try alternative test.
doPredictNextStateUtilities test passed.
Testing doComputeExpectedQValues...
doComputeExpectedQValues test passed.
Testing doComputeLoss...
doComputeLoss test passed.


# Training

Edit the globals as necessary.
The best model produced during training will be saved to ```SAVE_FILENAME``` with a number appended to the end.

###### Train for EASY_LEVEL

In [ ]:
SAVE_FILENAME = 'easy.model'
LOAD_FILENAME = None
TRAIN_SEED = EASY_LEVEL
policy_net = train(num_episodes=NUM_EPISODES, 
                   load_filename=LOAD_FILENAME, 
                   save_filename=SAVE_FILENAME, 
                   eval_interval=EVAL_INTERVAL, 
                   replay_capacity=REPLAY_CAPACITY, 
                   bootstrap_threshold=BOOTSTRAP, 
                   target_update=TARGET_UPDATE,
                   epsilon=EPSILON, 
                   eval_epsilon=EVAL_EPSILON,
                   gamma=GAMMA, 
                   batch_size=BATCH_SIZE,
                   random_seed=RANDOM_SEED,
                   seed=TRAIN_SEED)

Streaming output truncated to the last 5000 lines.
duration: 730
max reward: [107.00568]
result: coin
total steps: 43387 

episode: 57 epsilon: 0.6445386330969374
duration: 312
max reward: [108.]
result: coin
total steps: 43699 

episode: 58 epsilon: 0.6425308064000709
duration: 1002
max reward: [3.172204]
result: timeout
total steps: 44701 

episode: 59 epsilon: 0.6361247954319745
duration: 633
max reward: [107.08586]
result: coin
total steps: 45334 

episode: 60 epsilon: 0.6321108430390927
duration: 221
max reward: [108.]
result: coin
total steps: 45555 

Evaluating...
duration: 1002
max reward: [0.22687292]
result: timeout 

Evaluating...
duration: 1002
max reward: [0.05909991]
result: timeout 

Evaluating...
duration: 1002
max reward: [0.49504018]
result: timeout 

Evaluating...
duration: 1002
max reward: [0.30593014]
result: timeout 

Evaluating...
duration: 1002
max reward: [0.20593023]
result: timeout 

Evaluating...
duration: 1002
max reward: [0.05909991]
result: timeout 

Eval

###### Train for MEDIUM_LEVEL

In [ ]:
SAVE_FILENAME = 'medium.model'
LOAD_FILENAME = None
TRAIN_SEED = MEDIUM_LEVEL
policy_net = train(num_episodes=NUM_EPISODES, 
                   load_filename=LOAD_FILENAME, 
                   save_filename=SAVE_FILENAME, 
                   eval_interval=EVAL_INTERVAL, 
                   replay_capacity=REPLAY_CAPACITY, 
                   bootstrap_threshold=BOOTSTRAP, 
                   target_update=TARGET_UPDATE,
                   epsilon=EPSILON, 
                   eval_epsilon=EVAL_EPSILON,
                   gamma=GAMMA, 
                   batch_size=BATCH_SIZE,
                   random_seed=RANDOM_SEED,
                   seed=TRAIN_SEED)

Streaming output truncated to the last 5000 lines.
result: timeout
total steps: 51911 

episode: 57 epsilon: 0.5918745853230314
duration: 1002
max reward: [8.742842]
result: timeout
total steps: 52913 

episode: 58 epsilon: 0.5859736152099253
duration: 1002
max reward: [9.520224]
result: timeout
total steps: 53915 

episode: 59 epsilon: 0.580131477574407
duration: 1002
max reward: [9.385599]
result: timeout
total steps: 54917 

episode: 60 epsilon: 0.5743475858586133
duration: 1002
max reward: [12.22827]
result: timeout
total steps: 55919 

Evaluating...
duration: 1002
max reward: [4.260186]
result: timeout 

Evaluating...
duration: 1002
max reward: [3.7580118]
result: timeout 

Evaluating...
duration: 1002
max reward: [4.091881]
result: timeout 

Evaluating...
duration: 1002
max reward: [2.2748854]
result: timeout 

Evaluating...
duration: 1002
max reward: [1.7917943]
result: timeout 

Evaluating...
duration: 1002
max reward: [4.6477237]
result: timeout 

Evaluating...
duration: 1002


###### Train for ONE_MONSTER Level

In [ ]:
SAVE_FILENAME = 'monster.model'
LOAD_FILENAME = None
TRAIN_SEED = ONE_MONSTER
policy_net = train(num_episodes=NUM_EPISODES, 
                   load_filename=LOAD_FILENAME, 
                   save_filename=SAVE_FILENAME, 
                   eval_interval=EVAL_INTERVAL, 
                   replay_capacity=REPLAY_CAPACITY, 
                   bootstrap_threshold=BOOTSTRAP, 
                   target_update=TARGET_UPDATE,
                   epsilon=EPSILON, 
                   eval_epsilon=EVAL_EPSILON,
                   gamma=GAMMA, 
                   batch_size=BATCH_SIZE,
                   random_seed=RANDOM_SEED,
                   seed=TRAIN_SEED)

screen size:  40 40
Making new model.
training...
episode: 0 epsilon: 0.9


<ipython-input-15-bfb86c3d4208>:92: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  reward = torch.tensor([reward], device=DEVICE)


Streaming output truncated to the last 5000 lines.
result: died
total steps: 34902 

episode: 57 epsilon: 0.7016149598615509
duration: 356
max reward: [6.0186205]
result: died
total steps: 35258 

episode: 58 epsilon: 0.6991216513269012
duration: 27
max reward: [5.2530313]
result: died
total steps: 35285 

episode: 59 epsilon: 0.6989329139617337
duration: 1002
max reward: [3.3073854]
result: timeout
total steps: 36287 

episode: 60 epsilon: 0.6919645758397255
duration: 125
max reward: [5.3029003]
result: died
total steps: 36412 

Evaluating...
duration: 37
max reward: [5.6376033]
result: died 

Evaluating...
duration: 154
max reward: [6.0324373]
result: died 

Evaluating...
duration: 26
max reward: [108.]
result: coin 

Evaluating...
duration: 57
max reward: [5.6562333]
result: died 

Evaluating...
duration: 33
max reward: [5.584092]
result: died 

Evaluating...
duration: 27
max reward: [108.]
result: coin 

Evaluating...
duration: 23
max reward: [6.2698717]
result: died 

Evaluating..

# Evaluation

This loads the saved model (even if there is a number appended to the end) and evaluates it.

It also produces a dump of screenshots for each run of ```evaluate()``` in ```/content/cache/```.

###### Evaluate the easy.model

In [ ]:
TEST_FILENAME = 'easy.model'
TEST_SEED = EASY_LEVEL
eval_net = load_model(TEST_FILENAME)

for _ in range(EVAL_COUNT):
  duration, max_reward = evaluate(eval_net, epsilon=EVAL_EPSILON, test_seed=TEST_SEED)

Loading saved_models/easy.model.170 ...
Done loading.
Evaluating...
duration: 44
max reward: [107.13231]
result: coin 

Evaluating...
duration: 25
max reward: [106.806076]
result: coin 

Evaluating...
duration: 33
max reward: [106.81758]
result: coin 

Evaluating...
duration: 38
max reward: [106.93531]
result: coin 

Evaluating...
duration: 50
max reward: [107.16231]
result: coin 

Evaluating...
duration: 52
max reward: [108.]
result: coin 

Evaluating...
duration: 145
max reward: [108.]
result: coin 

Evaluating...
duration: 22
max reward: [107.00875]
result: coin 

Evaluating...
duration: 108
max reward: [108.]
result: coin 

Evaluating...
duration: 44
max reward: [107.16231]
result: coin 



###### Evaluate the medium.model

In [ ]:
TEST_FILENAME = 'medium.model'
TEST_SEED = MEDIUM_LEVEL
eval_net = load_model(TEST_FILENAME)

for _ in range(EVAL_COUNT):
  duration, max_reward = evaluate(eval_net, epsilon=EVAL_EPSILON, test_seed=TEST_SEED)

Loading saved_models/medium.model.380 ...
Done loading.
Evaluating...
duration: 114
max reward: [119.]
result: coin 

Evaluating...
duration: 602
max reward: [118.3721]
result: coin 

Evaluating...
duration: 118
max reward: [119.]
result: coin 

Evaluating...
duration: 86
max reward: [119.]
result: coin 

Evaluating...
duration: 153
max reward: [118.352776]
result: coin 

Evaluating...
duration: 439
max reward: [119.]
result: coin 

Evaluating...
duration: 206
max reward: [118.8987]
result: coin 

Evaluating...
duration: 272
max reward: [119.]
result: coin 

Evaluating...
duration: 108
max reward: [119.]
result: coin 

Evaluating...
duration: 81
max reward: [118.0763]
result: coin 



###### Evaluate the monster.model

In [ ]:
TEST_FILENAME = 'monster.model'
TEST_SEED = ONE_MONSTER
eval_net = load_model(TEST_FILENAME)

for _ in range(EVAL_COUNT):
  duration, max_reward = evaluate(eval_net, epsilon=EVAL_EPSILON, test_seed=TEST_SEED)

Loading saved_models/monster.model.460 ...
Done loading.
Evaluating...
duration: 46
max reward: [5.4868336]
result: died 

Evaluating...
duration: 39
max reward: [107.20856]
result: coin 

Evaluating...
duration: 39
max reward: [107.40057]
result: coin 

Evaluating...
duration: 27
max reward: [107.06554]
result: coin 

Evaluating...
duration: 31
max reward: [107.93341]
result: coin 

Evaluating...
duration: 25
max reward: [6.6005507]
result: died 

Evaluating...
duration: 26
max reward: [106.87934]
result: coin 

Evaluating...
duration: 43
max reward: [107.25714]
result: coin 

Evaluating...
duration: 38
max reward: [5.87613]
result: died 

Evaluating...
duration: 22
max reward: [6.2906632]
result: died 



#GRADING

###### Grade the easy.model

In [ ]:
TEST_FILENAME = 'easy.model'
TEST_SEED = EASY_LEVEL
eval_net = load_model(TEST_FILENAME)
best_easy_duration = None
best_easy_grade = 0.0

try:
  for i in range(10):
    print("EASY LEVEL TRY", i)
    easy_grade = 0.0
    total_duration = 0.0
    for j in range(10):
      duration, max_reward = evaluate(eval_net, epsilon = EVAL_EPSILON, test_seed = TEST_SEED)
      status, score = episode_status(duration, max_reward)
      total_duration = total_duration + score
    average_duration = total_duration / 10.0
    if average_duration < 150:
      easy_grade = easy_grade + 3
    if average_duration < 100:
      easy_grade = easy_grade + 1
    if average_duration < 50:
      easy_grade = easy_grade + 1
    if easy_grade > best_easy_grade:
      best_easy_grade = easy_grade
    if best_easy_duration is None or average_duration < best_easy_duration:
      best_easy_duration = average_duration
    print("EASY SCORE", i, easy_grade, average_duration)
    if easy_grade >= 5.0:
      break
except Exception as e: 
  print("EASY GRADING BROKE")

print("BEST EASY SCORE", best_easy_grade, best_easy_duration)

Loading saved_models/easy.model.170 ...
Done loading.
EASY LEVEL TRY 0
Evaluating...
duration: 50
max reward: [107.0753]
result: coin 

Evaluating...
duration: 31
max reward: [107.13148]
result: coin 

Evaluating...
duration: 22
max reward: [106.92502]
result: coin 

Evaluating...
duration: 58
max reward: [107.16231]
result: coin 

Evaluating...
duration: 64
max reward: [108.]
result: coin 

Evaluating...
duration: 52
max reward: [106.82183]
result: coin 

Evaluating...
duration: 41
max reward: [106.979095]
result: coin 

Evaluating...
duration: 52
max reward: [108.]
result: coin 

Evaluating...
duration: 130
max reward: [108.]
result: coin 

Evaluating...
duration: 25
max reward: [107.12547]
result: coin 

EASY SCORE 0 4.0 52.5
EASY LEVEL TRY 1
Evaluating...
duration: 143
max reward: [107.16231]
result: coin 

Evaluating...
duration: 62
max reward: [108.]
result: coin 

Evaluating...
duration: 43
max reward: [107.16231]
result: coin 

Evaluating...
duration: 32
max reward: [107.16231]

###### Grade the medium.model

In [ ]:
TEST_FILENAME = 'medium.model'
TEST_SEED = MEDIUM_LEVEL
eval_net = load_model(TEST_FILENAME)
best_medium_duration = None
best_medium_grade = 0.0

try:
  for i in range(10):
    print("MEDIUM LEVEL TRY", i)
    medium_grade = 0.0
    total_duration = 0.0
    for j in range(10):
      duration, max_reward = evaluate(eval_net, epsilon = EVAL_EPSILON, test_seed = TEST_SEED)
      status, score = episode_status(duration, max_reward)
      total_duration = total_duration + score
    average_duration = total_duration / 10.0
    if average_duration < 150:
      medium_grade = medium_grade + 1
    if medium_grade > best_medium_grade:
      best_medium_grade = medium_grade
    if best_medium_duration is None or average_duration < best_medium_duration:
      best_medium_duration = average_duration
    print("MEDIUM SCORE", i, medium_grade, average_duration)
    if medium_grade >= 1.0:
      break
except Exception as e: 
  print("MEDIUM GRADING BROKE")

print("BEST MEDIUM SCORE", best_medium_grade, best_medium_duration)

Loading saved_models/medium.model.380 ...
Done loading.
MEDIUM LEVEL TRY 0
Evaluating...
duration: 114
max reward: [118.97]
result: coin 

Evaluating...
duration: 155
max reward: [118.38433]
result: coin 

Evaluating...
duration: 68
max reward: [118.07838]
result: coin 

Evaluating...
duration: 102
max reward: [117.96403]
result: coin 

Evaluating...
duration: 160
max reward: [119.]
result: coin 

Evaluating...
duration: 149
max reward: [118.169266]
result: coin 

Evaluating...
duration: 203
max reward: [119.]
result: coin 

Evaluating...
duration: 84
max reward: [118.552895]
result: coin 

Evaluating...
duration: 70
max reward: [117.93533]
result: coin 

Evaluating...
duration: 88
max reward: [118.97]
result: coin 

MEDIUM SCORE 0 1.0 119.3
BEST MEDIUM SCORE 1.0 119.3


###### Grade the monster.model

In [ ]:
TEST_FILENAME = 'monster.model'
TEST_SEED = ONE_MONSTER
eval_net = load_model(TEST_FILENAME)
best_monster_duration = None
best_monster_grade = 0.0

try:
  for i in range(10):
    print("MONSTER LEVEL TRY", i)
    monster_grade = 0.0
    total_duration = 0.0
    for j in range(10):
      duration, max_reward = evaluate(eval_net, epsilon = EVAL_EPSILON, test_seed = TEST_SEED)
      status, score = episode_status(duration, max_reward)
      total_duration = total_duration + score
    average_duration = total_duration / 10.0
    if average_duration < 300:
      monster_grade = monster_grade + 1
    if monster_grade > best_monster_grade:
      best_monster_grade = monster_grade
    if best_monster_duration is None or average_duration < best_monster_duration:
      best_monster_duration = average_duration
    print("MONSTER SCORE", i, monster_grade, average_duration)
    if monster_grade >= 1.0:
      break
except Exception as e: 
  print("MONSTER GRADING BROKE")

print("BEST MONSTER SCORE", best_monster_grade, best_monster_duration)

Loading saved_models/monster.model.460 ...
Done loading.
MONSTER LEVEL TRY 0
Evaluating...
duration: 38
max reward: [5.87613]
result: died 

Evaluating...
duration: 39
max reward: [107.40057]
result: coin 

Evaluating...
duration: 66
max reward: [5.0165167]
result: died 

Evaluating...
duration: 46
max reward: [108.]
result: coin 

Evaluating...
duration: 40
max reward: [107.40057]
result: coin 

Evaluating...
duration: 55
max reward: [5.93013]
result: died 

Evaluating...
duration: 45
max reward: [107.605225]
result: coin 

Evaluating...
duration: 21
max reward: [5.614759]
result: died 

Evaluating...
duration: 43
max reward: [107.40057]
result: coin 

Evaluating...
duration: 41
max reward: [107.40057]
result: coin 

MONSTER SCORE 0 0.0 425.4
MONSTER LEVEL TRY 1
Evaluating...
duration: 22
max reward: [5.3265867]
result: died 

Evaluating...
duration: 29
max reward: [5.913707]
result: died 

Evaluating...
duration: 47
max reward: [107.40057]
result: coin 

Evaluating...
duration: 43
ma

# Download The Model

If you want to download the model to a computer without NVIDIA GPU and CUDA support, you can use the cell below. It takes the model indicated, converts it to run on CPU, and saves it. You can then open the file browser to the left and download the model file.<br/> <br/><b>Change the model name in load_model argument below to download medium and monster models respectively.</b>

In [ ]:
### USE THIS TO COVERT A MODEL FROM CUDA TO CPU
### DOWNLOAD USING THE FILES MENU TO THE LEFT

### CHANGE THE MODEL NAME BELOW FOR MEDIUM AND MONSTER MODELS
net = load_model('easy.model')
net = net.to('cpu')
torch.save(net, os.path.join(MODEL_PATH, 'saved_cpu.model'))

# Show Evaluation Movies

The following cells will render an animation in the notebook. It is flickery but you should get an idea of what your agent was duing. 

In [ ]:
# This seems to work but is really slow and flickery

eval_run_number = 1
show_movie(eval_run_number)

KeyboardInterrupt: ignored

Alternatively, the following makes an animated gif. The animation is faster and doesn't flicker once loaded. Unfortunately, colab loads animated gifs really slow.
Better to download and view on one's own machine (though this is cumbersome).

In [ ]:
eval_run_number = 1
make_anim(eval_run_number)